### learning-AI101 : topic4_3
**작곡 AI 나 혼자서 구현해보기**

#### 0. 데이터 전처리, 데이터 준비

In [3]:
data = open('../data/pianoabc/pianoabc.txt', 'r').read()
print (data[:10])

e"G"d2B"D"


In [4]:
# 여기서 data는 원본데이터로, 문자열로 되어있을 것임
print (type(data) == str)

True


In [5]:
# bag_of_words를 이용해서 data를 수치화하자
from collections import defaultdict

set_data = list(set(data))
set_data.sort() # 해시를 이용한 set은 지금 뒤죽박죽이니 sort()를 사용해서 정렬하자

bag_of_words = defaultdict()
ind = 0

for component in set_data : # bag_of_words를 만들기 위해 set_data를 순회하여 번호를 매기자
    bag_of_words[component] = ind
    ind += 1

print (bag_of_words)

defaultdict(None, {'"': 0, "'": 1, ',': 2, '/': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, ':': 13, 'A': 14, 'B': 15, 'C': 16, 'D': 17, 'E': 18, 'F': 19, 'G': 20, '^': 21, '_': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'z': 30})


In [6]:
# 이제 원본 데이터인 data를 bag_of_words를 이용하여 수치화한 후에 리스트에 저장해놓자

datalist = []

for word in data : 
    datalist.append(bag_of_words[word])

print (datalist[0:50])

[27, 0, 20, 0, 26, 5, 15, 0, 17, 0, 14, 6, 3, 5, 15, 3, 5, 25, 0, 20, 0, 15, 5, 20, 0, 17, 0, 14, 5, 27, 0, 20, 0, 26, 5, 15, 0, 17, 0, 14, 6, 3, 5, 15, 3, 5, 25, 13, 5, 3]


In [7]:
# train_X, train_Y 데이터를 만들자
# train_X의 component 갯수는 25, 그리고 train_Y의 component 갯수는 1 (train_X 다음의 component가 Y값이 될 것임)

train_X = []
train_Y = []
ind = 0

while True :
    try :
        train_X.append(datalist[ind:ind+25]) # 25개의 데이터
        train_Y.append(datalist[ind+25]) # 그 다음의 데이터
        ind += 25

    except IndexError : 
        break

print ('train_X 데이터 하나의 component : ', len(train_X[0]))
print ('train_X 데이터 갯수, train_Y 데이터 갯수 : ', len(train_X), len(train_Y))
# train_X의 데이터가 하나 더 많으니 하나를 pop
train_X.pop()
print (len(train_X) == len(train_Y)) # 갯수 맞추기 성공

train_X 데이터 하나의 component :  25
train_X 데이터 갯수, train_Y 데이터 갯수 :  11681 11680
True


In [8]:
import tensorflow as tf
import numpy as np

train_X = tf.one_hot(train_X, 31)
train_Y = tf.one_hot(train_Y, 31)
print(train_X[0:2])

tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(2, 25, 31), dtype=float32)


--------

#### 1. 모델링

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(200, input_shape=(25, 31)),
    tf.keras.layers.Dense(31, activation="softmax")
])

#### 2. 컴파일

In [10]:
model.compile (loss='categorical_crossentropy', optimizer='adam')

#### 3. fit


In [11]:
model.fit(train_X, train_Y, batch_size=64, epochs=50)

Epoch 1/50
183/183 [==============================] - 7s 28ms/step - loss: 2.5782
Epoch 2/50
183/183 [==============================] - 11s 58ms/step - loss: 2.0786
Epoch 3/50
183/183 [==============================] - 13s 73ms/step - loss: 1.8086
Epoch 4/50
183/183 [==============================] - 16s 88ms/step - loss: 1.6915
Epoch 5/50
183/183 [==============================] - 16s 85ms/step - loss: 1.6363
Epoch 6/50
183/183 [==============================] - 19s 104ms/step - loss: 1.5734
Epoch 7/50
183/183 [==============================] - 21s 114ms/step - loss: 1.5281
Epoch 8/50
183/183 [==============================] - 16s 88ms/step - loss: 1.4795
Epoch 9/50
183/183 [==============================] - 16s 86ms/step - loss: 1.4375
Epoch 10/50
183/183 [==============================] - 15s 83ms/step - loss: 1.4011
Epoch 11/50
183/183 [==============================] - 17s 95ms/step - loss: 1.3640
Epoch 12/50
183/183 [==============================] - 18s 99ms/step - loss: 1.3250


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 200)               185600    
                                                                 
 dense (Dense)               (None, 31)                6231      
                                                                 
Total params: 191831 (749.34 KB)
Trainable params: 191831 (749.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


------

#### 4. predict


In [21]:
# predict 100번 반복하여 정답율 확인해보기

import random

counting_accord = 0

for counting in range (100) :
    ind = random.randint(1, len(datalist) - 26)
    
    test_X = tf.one_hot(datalist[ind:ind+25], 31) # 원핫인코딩
    test_X = tf.expand_dims(test_X, axis=0) # 차원 하나 더 늘리기 (input과 shape를 맞춰주기 위해서)
    output = model.predict(test_X)
    
    if (np.argmax(output[0]) == datalist[ind+25]) :
        counting_accord += 1
        
    accuracy = (counting_accord / (counting + 1)) * 100
    
    print(accuracy)

1/1 [==============================] - 0s 17ms/step
100.0
1/1 [==============================] - 0s 17ms/step
100.0
1/1 [==============================] - 0s 16ms/step
100.0
1/1 [==============================] - 0s 17ms/step
75.0
1/1 [==============================] - 0s 17ms/step
60.0
1/1 [==============================] - 0s 17ms/step
66.66666666666666
1/1 [==============================] - 0s 17ms/step
71.42857142857143
1/1 [==============================] - 0s 17ms/step
62.5
1/1 [==============================] - 0s 17ms/step
55.55555555555556
1/1 [==============================] - 0s 17ms/step
50.0
1/1 [==============================] - 0s 18ms/step
54.54545454545454
1/1 [==============================] - 0s 18ms/step
50.0
1/1 [==============================] - 0s 18ms/step
53.84615384615385
1/1 [==============================] - 0s 19ms/step
57.14285714285714
1/1 [==============================] - 0s 17ms/step
60.0
1/1 [==============================] - 0s 17ms/step
62.5
1/1 [==